In [ ]:
# type: ignore
# Cell 1: Imports and Setup
import chess
import chess.pgn
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from pathlib import Path
import pyarrow as pa
import pyarrow.parquet as pq
from typing import List, Tuple
import multiprocessing as mp
from tqdm import tqdm
import gc
import h5py
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor  # Add ThreadPoolExecutor
import pickle
import sys
import os
import json
from datetime import datetime


from to_tensor import create_tensor, NUM_PLANES

# Configuration
CHUNK_SIZE = 50_000  # Games per chunk (324 chunks for 162M games)
MAX_CP = 1000
EMBEDDING_DIM = 256
NUM_CORES = 12  # Your CPU cores
FILE_PATH = "../../data/raw/lichess_eval/lichess_db_eval.jsonl"
OUTPUT_DIR = "../../data/preprocessed/lichess_preprocessed_chunks"
NUM_CORES = 12
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


LEDGER_FILE = Path("../../data/preprocessed/lichess_preprocessed_chunks/processing_ledger.json")

# Initialize the ThreadPoolExecutor for asynchronous file saving
save_executor = ThreadPoolExecutor(max_workers=1)  # Add this line!

print(f"Data source: {FILE_PATH}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Using tensor planes: {NUM_PLANES}")
print(f"Chunk size: {CHUNK_SIZE:,} positions per chunk")
print(f"Ledger file: {LEDGER_FILE}")

Data source: ../../data/raw/lichess_eval/lichess_db_eval.jsonl
Output directory: ../../data/preprocessed/lichess_preprocessed_chunks
Using tensor planes: 28
Chunk size: 50,000 positions per chunk
Ledger file: ..\..\data\preprocessed\lichess_preprocessed_chunks\processing_ledger.json


In [2]:
if not os.path.exists(FILE_PATH):
    raise FileNotFoundError(f"Input file not found: {FILE_PATH}")

print(f"✓ Input file found: {FILE_PATH}")
file_size_gb = os.path.getsize(FILE_PATH) / 1024 / 1024 / 1024
print(f"File size: {file_size_gb:.2f} GB")

# Initialize or load existing ledger
def load_ledger():
    """Load existing processing ledger or create new one"""
    if LEDGER_FILE.exists():
        with open(LEDGER_FILE, 'r') as f:
            ledger = json.load(f)
        print(f"✓ Loaded existing ledger: {len(ledger['completed_chunks'])} chunks already processed")
        return ledger
    else:
        ledger = {
            'source_file': FILE_PATH,
            'file_size_bytes': os.path.getsize(FILE_PATH),
            'chunk_size': CHUNK_SIZE,
            'num_planes': NUM_PLANES,
            'max_cp': MAX_CP,
            'started_at': datetime.now().isoformat(),
            'last_updated': datetime.now().isoformat(),
            'completed_chunks': [],
            'failed_chunks': [],
            'total_positions_processed': 0,
            'last_processed_line': 0,
            'status': 'initialized'
        }
        print("✓ Created new processing ledger")
        return ledger

def save_ledger(ledger):
    """Save ledger to disk"""
    ledger['last_updated'] = datetime.now().isoformat()
    with open(LEDGER_FILE, 'w') as f:
        json.dump(ledger, f, indent=2)

# Load ledger
processing_ledger = load_ledger()
print(f"Resume from line: {processing_ledger['last_processed_line']}")
print(f"Total processed so far: {processing_ledger['total_positions_processed']:,} positions")


✓ Input file found: ../../data/raw/lichess_eval/lichess_db_eval.jsonl
File size: 61.14 GB
✓ Loaded existing ledger: 614 chunks already processed
Resume from line: 35920709
Total processed so far: 30,750,000 positions


In [ ]:
# type: ignore
class ChessPositionProcessor:
    def __init__(self):
        pass
        
    def tensor_to_transformer_input(self, chess_tensor: torch.Tensor) -> np.ndarray:
        """Convert 28x8x8 chess tensor to transformer input format (64, 28)"""
        if chess_tensor.is_cuda:
            chess_tensor = chess_tensor.cpu()
            
        if isinstance(chess_tensor, torch.Tensor):
            chess_tensor = chess_tensor.numpy()
            
        # Reshape from (28, 8, 8) to (28, 64)
        tensor_flat = chess_tensor.reshape(NUM_PLANES, 64)
        
        # Transpose to get 64 tokens with 28 features each
        square_tokens = tensor_flat.T  # (64, 28)
        
        return square_tokens.astype(np.float32)
    
    def create_position_data(self, square_tokens: np.ndarray, evaluation: float, metadata: dict) -> dict:
        """Create complete position data"""
        return {
            'square_tokens': square_tokens,
            'evaluation': evaluation,
            'raw_cp': metadata.get('raw_cp', 0),
            'clipped_cp': metadata.get('clipped_cp', 0),
            'fen': metadata.get('fen', ''),
            'line_number': metadata.get('line_number', 0)
        }

def process_jsonl_line(line: str, line_number: int, processor: ChessPositionProcessor):
    """Process a single JSONL line and return position data or None"""
    try:
        obj = json.loads(line.strip())
        fen = obj.get('fen')
        cp = None

        # Extract centipawn evaluation
        if obj.get('evals') and len(obj['evals']) > 0:
            if obj['evals'][0].get('pvs') and len(obj['evals'][0]['pvs']) > 0:
                cp_value = obj['evals'][0]['pvs'][0].get('cp')
                if cp_value is not None:
                    cp = int(cp_value)

        if fen is not None and cp is not None:
            try:
                board = chess.Board(fen)
                
                # Create tensor using your function
                chess_tensor = create_tensor(board)
                
                # Convert to transformer input
                transformer_input = processor.tensor_to_transformer_input(chess_tensor)
                
                # Clip and normalize evaluation
                clipped_cp = max(min(cp, MAX_CP), -MAX_CP)
                normalized_evaluation = clipped_cp / MAX_CP
                
                # Create metadata
                metadata = {
                    'raw_cp': cp,
                    'clipped_cp': clipped_cp,
                    'fen': fen,
                    'line_number': line_number
                }
                
                return processor.create_position_data(transformer_input, normalized_evaluation, metadata)
                
            except (ValueError, Exception):
                return None
    except (json.JSONDecodeError, Exception):
        return None
    
    return None

print("✓ Data processing functions ready")


✓ Data processing functions ready


In [4]:
def save_chunk(positions_data: List[dict], chunk_id: int) -> dict:
    """Save a chunk of positions to disk"""
    chunk_path = Path(OUTPUT_DIR) / f"chunk_{chunk_id:04d}.h5"
    
    with h5py.File(chunk_path, 'w') as f:
        # Store square tokens
        tokens_array = np.array([pos['square_tokens'] for pos in positions_data])
        f.create_dataset('square_tokens', data=tokens_array, compression='gzip', compression_opts=9)
        
        # Store evaluations
        evaluations = np.array([pos['evaluation'] for pos in positions_data], dtype=np.float32)
        f.create_dataset('evaluations', data=evaluations, compression='gzip')
        
        # Store metadata
        raw_cps = np.array([pos['raw_cp'] for pos in positions_data], dtype=np.int32)
        clipped_cps = np.array([pos['clipped_cp'] for pos in positions_data], dtype=np.int32)
        line_numbers = np.array([pos['line_number'] for pos in positions_data], dtype=np.int32)
        
        f.create_dataset('raw_cp', data=raw_cps)
        f.create_dataset('clipped_cp', data=clipped_cps)
        f.create_dataset('line_numbers', data=line_numbers)
        
        # Store FEN strings (variable length)
        fens = [pos['fen'].encode('utf-8') for pos in positions_data]
        dt = h5py.special_dtype(vlen=str)
        f.create_dataset('fens', data=[pos['fen'] for pos in positions_data], dtype=dt)
        
        # Store chunk metadata
        f.attrs['chunk_id'] = chunk_id
        f.attrs['num_positions'] = len(positions_data)
        f.attrs['num_planes'] = NUM_PLANES
        f.attrs['max_cp'] = MAX_CP
        f.attrs['source_file'] = os.path.basename(FILE_PATH)
        f.attrs['created_at'] = datetime.now().isoformat()
    
    file_size_mb = chunk_path.stat().st_size / 1024 / 1024
    
    return {
        'chunk_id': chunk_id,
        'num_positions': len(positions_data),
        'file_path': str(chunk_path),
        'file_size_mb': file_size_mb,
        'created_at': datetime.now().isoformat()
    }

def get_next_chunk_id(ledger):
    """Get the next chunk ID to process"""
    if ledger['completed_chunks']:
        return max(chunk['chunk_id'] for chunk in ledger['completed_chunks']) + 1
    else:
        return 0

print("✓ Chunk saving functions ready")


✓ Chunk saving functions ready


In [ ]:
# type: ignore
def process_file_from_resume_point():
    """Process the file starting from the last processed line"""
    
    processor = ChessPositionProcessor()
    current_chunk_data = []
    current_line = 0
    positions_processed_this_session = 0
    start_line = processing_ledger['last_processed_line']
    
    active_save_future = None # To keep track of the save operation for the current/previous chunk

    print(f"Starting processing from line {start_line + 1}")
    print(f"Target chunk size: {CHUNK_SIZE:,} positions")
    
    try:
        with open(FILE_PATH, 'r', encoding='utf-8') as file:
            # Skip already processed lines
            for _ in range(start_line):
                next(file)
                current_line += 1
            
            # Process remaining lines
            for line in tqdm(file, desc="Processing positions", unit="lines", initial=start_line):
                current_line += 1
                
                # Process this line
                position_data = process_jsonl_line(line, current_line, processor)
                
                if position_data is not None:
                    current_chunk_data.append(position_data)
                    positions_processed_this_session += 1
                    
                    # Check if chunk is full
                    if len(current_chunk_data) >= CHUNK_SIZE:
                        # A chunk is ready. First, ensure the previous save (if any) is complete.
                        if active_save_future:
                            try:
                                chunk_info = active_save_future.result() # Wait for previous save
                                processing_ledger['completed_chunks'].append(chunk_info)
                                # Ledger already reflects total_positions and last_line from submission time.
                                # Here we just confirm completion.
                                save_ledger(processing_ledger) # Persist the completed chunk info
                                print(f"\n✓ Previous chunk {chunk_info['chunk_id']} saved. Ledger updated.")
                            except Exception as e:
                                associated_chunk_id = getattr(active_save_future, 'associated_chunk_id', 'unknown')
                                print(f"\n✗ Error saving previous chunk (ID: {associated_chunk_id}): {e}")
                                processing_ledger['failed_chunks'].append({
                                    'chunk_id': associated_chunk_id, 'error': str(e), 
                                    'timestamp': datetime.now().isoformat(),
                                    'last_processed_line_at_failure': processing_ledger['last_processed_line']
                                })
                                save_ledger(processing_ledger)
                            finally:
                                active_save_future = None
                        
                        # Prepare and submit the current chunk for saving
                        chunk_id = get_next_chunk_id(processing_ledger)
                        data_to_save = list(current_chunk_data) # Create a copy for the thread

                        print(f"\nSubmitting chunk {chunk_id} ({len(data_to_save)} positions) for asynchronous saving...")
                        active_save_future = save_executor.submit(save_chunk, data_to_save, chunk_id)
                        active_save_future.associated_chunk_id = chunk_id # Store for error reporting

                        # Optimistically update ledger for submitted chunk
                        processing_ledger['total_positions_processed'] += len(data_to_save)
                        processing_ledger['last_processed_line'] = current_line
                        processing_ledger['status'] = 'processing'
                        save_ledger(processing_ledger) # Save progress including this submitted chunk
                        
                        print(f"  Chunk {chunk_id} submitted. Main thread continues.")
                        print(f"  Total processed (optimistic): {processing_ledger['total_positions_processed']:,} positions")
                        print(f"  Progress: Line {current_line:,}")
                        
                        # Reset for next chunk
                        current_chunk_data = []
                        gc.collect()
                
                # Save progress every 100k lines
                if current_line % 100000 == 0 and current_line > start_line : # Avoid saving at the very start if resuming
                    # Periodic save of overall progress (last_processed_line)
                    # The main chunk saving logic already updates the ledger more frequently.
                    # This specific periodic save might be redundant or can be adjusted.
                    # For safety, ensure ledger reflects current line.
                    processing_ledger['last_processed_line'] = current_line
                    save_ledger(processing_ledger)
                    print(f"Progress checkpoint: Line {current_line:,}, Found {positions_processed_this_session:,} valid positions this session.")
        
        # After the loop, handle the last (potentially partial) chunk
        if current_chunk_data:
            if active_save_future: # Wait for any pending save from a full chunk
                try:
                    chunk_info = active_save_future.result()
                    processing_ledger['completed_chunks'].append(chunk_info)
                    save_ledger(processing_ledger)
                    print(f"\n✓ Previous chunk {chunk_info['chunk_id']} saved before final. Ledger updated.")
                except Exception as e:
                    associated_chunk_id = getattr(active_save_future, 'associated_chunk_id', 'unknown')
                    print(f"\n✗ Error saving previous chunk (ID: {associated_chunk_id}) before final: {e}")
                    processing_ledger['failed_chunks'].append({
                        'chunk_id': associated_chunk_id, 'error': str(e),
                        'timestamp': datetime.now().isoformat()
                    })
                    save_ledger(processing_ledger)
                finally:
                    active_save_future = None

            chunk_id = get_next_chunk_id(processing_ledger)
            data_to_save = list(current_chunk_data)
            print(f"\nSubmitting final chunk {chunk_id} ({len(data_to_save)} positions) for asynchronous saving...")
            active_save_future = save_executor.submit(save_chunk, data_to_save, chunk_id)
            active_save_future.associated_chunk_id = chunk_id

            processing_ledger['total_positions_processed'] += len(data_to_save)
            processing_ledger['last_processed_line'] = current_line
            save_ledger(processing_ledger) # Save progress for the submission of the final chunk

        # Wait for the very last submitted save operation to complete
        if active_save_future:
            try:
                chunk_info = active_save_future.result()
                processing_ledger['completed_chunks'].append(chunk_info)
                print(f"\n✓ Final chunk {chunk_info['chunk_id']} saved. Ledger updated.")
            except Exception as e:
                associated_chunk_id = getattr(active_save_future, 'associated_chunk_id', 'unknown')
                print(f"\n✗ Error saving final chunk (ID: {associated_chunk_id}): {e}")
                processing_ledger['failed_chunks'].append({
                    'chunk_id': associated_chunk_id, 'error': str(e),
                    'timestamp': datetime.now().isoformat()
                })
            finally:
                active_save_future = None
        
        processing_ledger['status'] = 'completed'
        processing_ledger['completed_at'] = datetime.now().isoformat()
        save_ledger(processing_ledger)
        
        print(f"\n" + "="*60)
        print(f"PROCESSING COMPLETED!")
        print(f"Total chunks created: {len(processing_ledger['completed_chunks'])}")
        print(f"Total positions processed: {processing_ledger['total_positions_processed']:,}")
        print(f"Total lines processed: {current_line:,}")
        print(f"Positions this session: {positions_processed_this_session:,}")
        
        total_size_mb = sum(chunk['file_size_mb'] for chunk in processing_ledger['completed_chunks'])
        print(f"Total storage used: {total_size_mb / 1024:.2f} GB")
        print(f"="*60)
        
    except KeyboardInterrupt:
        print(f"\n⚠ Processing interrupted by user")
        processing_ledger['last_processed_line'] = current_line
        processing_ledger['status'] = 'interrupted'
        save_ledger(processing_ledger)
        print(f"Progress saved. Can resume from line {current_line + 1}") # +1 if current_line is 0-indexed last processed
        
    except Exception as e:
        print(f"\n✗ Error during processing: {e}")
        processing_ledger['last_processed_line'] = current_line
        processing_ledger['status'] = 'error'
        processing_ledger['last_error'] = str(e)
        save_ledger(processing_ledger)
        raise
    finally:
        print("Shutting down save executor...")
        save_executor.shutdown(wait=True) # Ensure all pending save tasks are completed
        print("Save executor shut down.")

print("✓ Main processing function ready")

✓ Main processing function ready


In [ ]:
# type: ignore
def show_processing_status():
    """Display current processing status"""
    ledger = load_ledger()
    
    print("="*50)
    print("PROCESSING STATUS")
    print("="*50)
    print(f"Source file: {os.path.basename(ledger['source_file'])}")
    print(f"File size: {ledger['file_size_bytes'] / 1024 / 1024 / 1024:.2f} GB")
    print(f"Status: {ledger['status'].upper()}")
    print(f"Started: {ledger['started_at']}")
    print(f"Last updated: {ledger['last_updated']}")
    print()
    print(f"Chunks completed: {len(ledger['completed_chunks'])}")
    print(f"Positions processed: {ledger['total_positions_processed']:,}")
    print(f"Last processed line: {ledger['last_processed_line']:,}")
    
    if ledger['completed_chunks']:
        total_size = sum(chunk['file_size_mb'] for chunk in ledger['completed_chunks'])
        avg_positions_per_chunk = ledger['total_positions_processed'] / len(ledger['completed_chunks'])
        print(f"Total storage: {total_size / 1024:.2f} GB")
        print(f"Avg positions per chunk: {avg_positions_per_chunk:,.0f}")
    
    if ledger['failed_chunks']:
        print(f"Failed chunks: {len(ledger['failed_chunks'])}")
    
    if 'last_error' in ledger:
        print(f"Last error: {ledger['last_error']}")
    
    print("="*50)

def list_completed_chunks():
    """List all completed chunks"""
    ledger = load_ledger()
    
    if not ledger['completed_chunks']:
        print("No chunks completed yet")
        return
    
    chunks_df = pd.DataFrame(ledger['completed_chunks'])
    chunks_df['file_size_mb'] = chunks_df['file_size_mb'].round(1)
    
    print("Completed chunks:")
    print(chunks_df[['chunk_id', 'num_positions', 'file_size_mb', 'created_at']])
    
    return chunks_df

def verify_chunk_files():
    """Verify that all chunk files exist and are readable"""
    ledger = load_ledger()
    missing_files = []
    corrupted_files = []
    
    print("Verifying chunk files...")
    
    for chunk in tqdm(ledger['completed_chunks'], desc="Checking chunks"):
        chunk_path = Path(chunk['file_path'])
        
        if not chunk_path.exists():
            missing_files.append(chunk['chunk_id'])
            continue
            
        try:
            with h5py.File(chunk_path, 'r') as f:
                # Quick verification
                tokens = f['square_tokens']
                evals = f['evaluations']
                
                if tokens.shape[0] != evals.shape[0]:
                    corrupted_files.append(chunk['chunk_id'])
                    
        except Exception as e:
            corrupted_files.append(chunk['chunk_id'])
    
    if missing_files:
        print(f"⚠ Missing chunk files: {missing_files}")
    
    if corrupted_files:
        print(f"⚠ Corrupted chunk files: {corrupted_files}")
    
    if not missing_files and not corrupted_files:
        print("✓ All chunk files verified successfully")
    
    return missing_files, corrupted_files

# Show current status
show_processing_status()


✓ Loaded existing ledger: 614 chunks already processed
PROCESSING STATUS
Source file: lichess_db_eval.jsonl
File size: 61.14 GB
Status: INTERRUPTED
Started: 2025-06-16T23:51:37.065746
Last updated: 2025-06-17T10:00:21.995777

Chunks completed: 614
Positions processed: 30,750,000
Last processed line: 35,920,709
Total storage: 5.13 GB
Avg positions per chunk: 50,081
Last error: name 'save_executor' is not defined


In [7]:
def start_processing():
    """Start or resume processing"""
    ledger = load_ledger()
    
    if ledger['status'] == 'completed':
        print("✓ Processing already completed!")
        show_processing_status()
        return
    
    print("Starting data processing...")
    print(f"Will process {FILE_PATH}")
    print(f"Resume from line: {ledger['last_processed_line'] + 1}")
    print(f"Output directory: {OUTPUT_DIR}")
    print()
    print("Press Ctrl+C to safely interrupt and save progress")
    print("-" * 50)
    
    try:
        process_file_from_resume_point()
    except Exception as e:
        print(f"Processing failed: {e}")
        return
    
    print("\n✓ Processing completed successfully!")
    print("Run verify_chunk_files() to verify all files")

# Uncomment the line below to start processing
# start_processing()

print("Ready to process!")
print("Run start_processing() to begin")


Ready to process!
Run start_processing() to begin


In [8]:
def print_quick_stats():
    """Print quick statistics about processed data"""
    ledger = load_ledger()
    
    if not ledger['completed_chunks']:
        print("No data processed yet")
        return
    
    # Calculate stats
    total_chunks = len(ledger['completed_chunks'])
    total_positions = ledger['total_positions_processed']
    total_size_gb = sum(chunk['file_size_mb'] for chunk in ledger['completed_chunks']) / 1024
    
    chunk_sizes = [chunk['num_positions'] for chunk in ledger['completed_chunks']]
    avg_chunk_size = np.mean(chunk_sizes)
    min_chunk_size = np.min(chunk_sizes)
    max_chunk_size = np.max(chunk_sizes)
    
    print("📊 PROCESSING SUMMARY")
    print("-" * 30)
    print(f"Total chunks: {total_chunks}")
    print(f"Total positions: {total_positions:,}")
    print(f"Total storage: {total_size_gb:.2f} GB")
    print(f"Avg chunk size: {avg_chunk_size:,.0f} positions")
    print(f"Min chunk size: {min_chunk_size:,} positions")
    print(f"Max chunk size: {max_chunk_size:,} positions")
    
    if total_positions > 0:
        compression_ratio = (total_positions * 64 * NUM_PLANES * 4) / (total_size_gb * 1024**3)
        print(f"Compression ratio: {compression_ratio:.1f}x")
    
    print(f"Status: {ledger['status']}")

# Print stats if data exists
print_quick_stats()

print("\n🎯 AVAILABLE COMMANDS:")
print("- start_processing()     : Start/resume processing")
print("- show_processing_status(): Show detailed status")  
print("- list_completed_chunks(): List all chunks")
print("- verify_chunk_files()   : Check file integrity")
print("- print_quick_stats()    : Show summary stats")


✓ Loaded existing ledger: 614 chunks already processed
📊 PROCESSING SUMMARY
------------------------------
Total chunks: 614
Total positions: 30,750,000
Total storage: 5.13 GB
Avg chunk size: 50,000 positions
Min chunk size: 50,000 positions
Max chunk size: 50,000 positions
Compression ratio: 40.0x
Status: interrupted

🎯 AVAILABLE COMMANDS:
- start_processing()     : Start/resume processing
- show_processing_status(): Show detailed status
- list_completed_chunks(): List all chunks
- verify_chunk_files()   : Check file integrity
- print_quick_stats()    : Show summary stats


In [9]:
start_processing()   

✓ Loaded existing ledger: 614 chunks already processed
Starting data processing...
Will process ../../data/raw/lichess_eval/lichess_db_eval.jsonl
Resume from line: 35920710
Output directory: ../../data/preprocessed/lichess_preprocessed_chunks

Press Ctrl+C to safely interrupt and save progress
--------------------------------------------------
Starting processing from line 35920710
Target chunk size: 50,000 positions


Processing positions: 35978253lines [00:46, 1046.29lines/s]


Submitting chunk 614 (50000 positions) for asynchronous saving...
  Chunk 614 submitted. Main thread continues.
  Total processed (optimistic): 30,800,000 positions
  Progress: Line 35,978,207


Processing positions: 36000144lines [01:04, 1166.31lines/s]

Progress checkpoint: Line 36,000,000, Found 69,059 valid positions this session.


...

Progress checkpoint: Line 80,400,000, Found 38,776,398 valid positions this session.


Processing positions: 80427398lines [10:20:54, 885.05lines/s] 


✓ Previous chunk 1388 saved. Ledger updated.

Submitting chunk 1389 (50000 positions) for asynchronous saving...
  Chunk 1389 submitted. Main thread continues.
  Total processed (optimistic): 69,550,000 positions
  Progress: Line 80,427,300


Processing positions: 80428090lines [10:20:55, 1194.65lines/s]



⚠ Processing interrupted by user
Progress saved. Can resume from line 80428092
Shutting down save executor...
Save executor shut down.

✓ Processing completed successfully!
Run verify_chunk_files() to verify all files
